In [ ]:
from simple_foraging_env import SimpleForagingEnv, SimpleAgent, RandomAgent
from opponent_model import OpponentModel, SubGoalSelector
from q_agent import QLearningAgent, ReplayBuffer
from q_agent_classic import QLearningAgentClassic
from omg_args import OMGArgs
import transformers as t
import matplotlib.pyplot as plt
import torch
import os
import random
import numpy as np
from collections import deque
from typing import Deque, Dict, List, Tuple, Optional

In [ ]:
from experiments.om import q_agent


env = SimpleForagingEnv(grid_size=7, max_steps=30)

obs_sample = env.reset()[0]
H, W, F_dim = obs_sample.shape
NUM_ACTIONS = 4

args = OMGArgs(
  device="cpu",
  folder_id="0",
  batch_size=32,
  horizon_H=6,
  qnet_hidden=256,
  max_steps=30,
  selector_mode="conservative",
  vae_beta=0.1,
  beta_start=1.0,
  beta_end=2.0,
  selector_tau_start=2.0,
  selector_tau_end=0.1,
  state_shape=obs_sample.shape,
  H=H, W=W,
  state_feature_splits=(F_dim,),
  action_dim=NUM_ACTIONS,
  latent_dim=32,
  d_model=256,
  nhead=4,
  num_encoder_layers=1,
  num_decoder_layers=1,
  dim_feedforward=1024,
  dropout=0.1,
)

os.makedirs("./diagrams_0", exist_ok=True)

vae = t.TransformerVAE(args)
cvae = t.TransformerCVAE(args)
vae.load_state_dict(torch.load('./models_0/vae.pth', map_location=args.device))
# cvae.load_state_dict(torch.load('./models_0/cvae.pth', map_location=args.device))
selector = SubGoalSelector(args)

op_model = OpponentModel(
  cvae, vae, selector, args=args)

q_agent = QLearningAgent(
    env=env,
    opponent_model=op_model,
    args=args
)
# q_agent.q.load_state_dict(torch.load('./models_0/qnet.pth', map_location=args.device))

In [ ]:
env._place_agent(0, (2, 0))
obs = env._get_observations()
op_model.prior_model.eval()
recon_logits, mu, logvar = op_model.prior_model(
    torch.from_numpy(obs[0]).float().unsqueeze(0).to(args.device)
)
op_model.visualize_subgoal_logits(
    obs[0], recon_logits)
print(mu.squeeze(0).detach().cpu().numpy())

In [ ]:
env._place_agent(0, (2, 1))
obs = env._get_observations()
op_model.prior_model.eval()
recon_logits, mu, logvar = op_model.prior_model(
    torch.from_numpy(obs[0]).float().unsqueeze(0).to(args.device)
)
op_model.visualize_subgoal_logits(
    obs[0], recon_logits)
print(mu.squeeze(0).detach().cpu().numpy())

In [ ]:
env._place_agent(0, (2, 2))
obs = env._get_observations()
op_model.prior_model.eval()
recon_logits, mu, logvar = op_model.prior_model(
    torch.from_numpy(obs[0]).float().unsqueeze(0).to(args.device)
)
op_model.visualize_subgoal_logits(
    obs[0], recon_logits)
print(mu.squeeze(0).detach().cpu().numpy())

In [ ]:
env._place_agent(0, (2, 3))
obs = env._get_observations()
op_model.prior_model.eval()
recon_logits, mu, logvar = op_model.prior_model(
    torch.from_numpy(obs[0]).float().unsqueeze(0).to(args.device)
)
op_model.visualize_subgoal_logits(
    obs[0], recon_logits)
print(mu.squeeze(0).detach().cpu().numpy())

In [14]:
projector = torch.randn(args.latent_dim, requires_grad=False)

In [15]:
def oracle_subgoal(x: torch.tensor):
  B, H, W, _ = x.shape
  g = torch.zeros((B, args.latent_dim))
  # opp start is at row 3, col 6
  opp_start = torch.tensor([3, 6]).float()
  for b in range(B):
    food_indices = (x[b, :, :, 1] == 1).nonzero(as_tuple=False)
    opp_idx = (x[b, :, :, 3] == 1).nonzero(as_tuple=False).float()

    is_top_food = 0.0
    # if we have food and opponent is not at start position
    if len(food_indices) > 0 and not torch.all(opp_idx[0] == opp_start):
      # find closest food to opponent
      dists = torch.norm(food_indices - opp_idx[0], dim=1)
      target_idx = torch.argmin(dists)
      target_row = food_indices[target_idx][0]
      # target_row is food that is closes to opponent

      # If target row is in upper half, it's Top (1.0), else Bottom (-1.0)
      is_top_food = 1.0 if target_row < H / 2 else -1.0
      print(f"Target row: {target_row.item()}, is_top_food: {is_top_food}")

    g[b] = is_top_food * projector

  return g, torch.zeros_like(g)

In [25]:
_ = env.reset()
env._place_agent(1, (4, 3))
# env.step({0: 0, 1: 2})
obs = env._get_observations()[0]
mu, logvar = oracle_subgoal(torch.from_numpy(obs).float().unsqueeze(0).to(args.device))
env.render_from_obs(obs)
print(mu)

Target row: 6, is_top_food: -1.0
. . . F . . .
. . . . . . .
. . . . . . .
A . . . . . .
. . . B . . .
. . . . . . .
. . . F . . .

tensor([[-0.5170,  0.1630, -1.1325,  0.5870, -0.2953,  0.9064,  0.4759, -0.3651,
         -0.3488, -0.7705, -0.9284,  1.4876,  0.1683, -1.1446, -0.0156, -0.1451,
          0.1154,  0.8681,  1.0336, -0.0344, -0.1319, -0.5101,  1.8010, -1.4599,
          0.4165,  0.3571,  0.8036, -1.3574,  0.4809,  1.3422,  0.3580, -0.7073]])


In [ ]:
action_names = {0: "UP", 1: "DOWN", 2: "LEFT", 3: "RIGHT"}
def _choose_action(qvals: torch.Tensor) -> int:
    qvals = qvals.squeeze(0)  # (A,)
    max_q = torch.max(qvals).item()
    max_actions = (qvals == max_q).nonzero(as_tuple=False).view(-1)
    if len(max_actions) > 1:
      return int(max_actions[torch.randint(len(max_actions), (1,))].item())
    return int(torch.argmax(qvals, dim=-1).item())

def select_action(s_t: np.ndarray, history: Dict[str, List[torch.Tensor]]) -> Tuple[int, torch.Tensor]:
  """
  (interaction phase) Infer g_hat and act eps-greedily on Q(s,g_hat,*)
  """
  ghat_mu, ghat_logvar = q_agent._infer_ghat(s_t, history)  # (1, latent_dim)
  s = torch.from_numpy(s_t).float().unsqueeze(0).to(args.device)
  qvals = q_agent.q(s, ghat_mu)
  print("\t\t\tUP DOWN LEFT RIGHT")
  print(f"Infered latent Q: {qvals.squeeze(0).detach().cpu().numpy()}")
  a = _choose_action(qvals)
  return a, ghat_mu.squeeze(0), ghat_logvar.squeeze(0)

def select_action_random_latent(s_t: np.ndarray) -> Tuple[int, torch.Tensor]:
  """
  (interaction phase) Infer g_hat and act eps-greedily on Q(s,g_hat,*)
  """
  ghat_mu = torch.randn(1, args.latent_dim).to(args.device)
  ghat_logvar = torch.randn(1, args.latent_dim).to(args.device)
  s = torch.from_numpy(s_t).float().unsqueeze(0).to(args.device)
  qvals = q_agent.q(s, ghat_mu)
  print(f"Random latent Q: {qvals.squeeze(0).detach().cpu().numpy()}")
  a = _choose_action(qvals)
  return a, ghat_mu.squeeze(0), ghat_logvar.squeeze(0)

In [ ]:
replay = ReplayBuffer(10_000)
opponent_agent = SimpleAgent(1)
  
for episode in range(100):
  opponent_agent.reset()
  obs = env.reset()
  done = False
  ep_ret = 0.0

  # History container
  history_len = args.max_history_length
  history = {
      "states": deque(maxlen=history_len),
      "actions": deque(maxlen=history_len)
  }

  step_buffer = deque(maxlen=args.horizon_H + 1)

  for step in range(args.max_steps or 500):
    # Convert deque to list for the model
    current_history = {k: list(v) for k, v in history.items()}

    a, ghat_mu, ghat_logvar = select_action(obs[0], current_history)
    a_opponent = opponent_agent.select_action(obs[1])
    actions = {0: a, 1: a_opponent}
    next_obs, reward, done, info = env.step(actions)

    # Store the current step's info
    step_info = {
        "state": obs[0].copy(),
        "action": a,
        "reward": float(reward[0]),
        "next_state": next_obs[0].copy(),
        "done": bool(done),
        "infer_mu": ghat_mu.detach().cpu(),
        "infer_log_var": ghat_logvar.detach().cpu(),
        "history": {k: [t.clone() for t in v] for k, v in current_history.items()}
    }
    step_buffer.append(step_info)

    # Once the buffer is full, the oldest step has its full future window
    if len(step_buffer) == args.horizon_H + 1:
      transition_to_store = step_buffer[0]
      future_states = [s["state"] for s in list(step_buffer)[1:]]
      transition_to_store["future_states"] = future_states
      replay.push(transition_to_store)
    elif done and len(step_buffer) > 1:
      # If episode ends, fill the future window with remaining states
      while len(step_buffer) > 1:
        transition_to_store = step_buffer.popleft()
        future_states = [s["state"] for s in list(step_buffer)]
        # Fill the rest with copies of the terminal state
        for _ in range(args.horizon_H - len(future_states)):
          future_states.append(step_buffer[-1]["state"])
        transition_to_store["future_states"] = future_states
        replay.push(transition_to_store)

    # Update history for the next step
    history["states"].append(torch.from_numpy(obs[0]).float())
    history["actions"].append(torch.tensor(a, dtype=torch.long))

    ep_ret += reward[0]
    obs = next_obs

    if done:
      break
  print(f"Episode {episode + 1} Return: {ep_ret}")
print("Data collection complete.")

In [ ]:
def run_episode():
  opponent_agent = SimpleAgent(1)
  obs = env.reset()
  done = False
  ep_ret = 0.0

  # History container
  history_len = args.max_history_length
  history = {
      "states": deque(maxlen=history_len),
      "actions": deque(maxlen=history_len)
  }

  for step in range(args.max_steps or 500):
    # Convert deque to list for the model
    current_history = {k: list(v) for k, v in history.items()}

    a, ghat_mu, ghat_logvar = select_action_random_latent(obs[0])
    a, ghat_mu, ghat_logvar = select_action(obs[0], current_history)
    a_opponent = opponent_agent.select_action(obs[1])
    actions = {0: a, 1: a_opponent}
    env.render_from_obs(obs[0])
    print("Selected action:", action_names[a])
    print(ghat_mu.detach().cpu().numpy())

    
    next_obs, reward, done, info = env.step(actions)

    # Update history for the next step
    history["states"].append(torch.from_numpy(obs[0]).float())
    history["actions"].append(torch.tensor(a, dtype=torch.long))

    ep_ret += reward[0]
    obs = next_obs

    if done:
      break

In [ ]:
run_episode()

In [ ]:
batch_list = replay.sample(1)
om_batch = {
        # States: (B, H, W, F)
        "states": torch.stack([torch.from_numpy(b["state"]).float() for b in batch_list], dim=0),
        "history": q_agent._collate_history([b["history"] for b in batch_list]),
        "future_states": torch.stack([torch.from_numpy(np.stack(b["future_states"])) for b in batch_list], dim=0),
        "infer_mu": torch.stack([b["infer_mu"] for b in batch_list], dim=0),
        "infer_log_var": torch.stack([b["infer_log_var"] for b in batch_list], dim=0),
        "dones": torch.tensor([b["done"] for b in batch_list], dtype=torch.float32, device=args.device)
    }
print(om_batch["states"].shape)
print(len(om_batch["history"]))
print(om_batch["future_states"].shape)
print(om_batch["infer_mu"].shape)
print(om_batch["infer_log_var"].shape)


In [ ]:
op_model.inference_model.eval()
with torch.no_grad():
    recon_logits, mu, logvar = op_model.inference_model(
        om_batch["states"],
        om_batch["history"],
    )
print("Reconstructed logits shape:", recon_logits.shape)
print("Inferred mu shape:", mu)
print("Inferred logvar shape:", logvar)
op_model.visualize_subgoal_logits(
    om_batch["states"][0].numpy(), recon_logits)